In [1]:
from functorch import vmap
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import *
import pathlib
import pickle
import einops
import math
from torchvision.models import resnet50, ResNet50_Weights, resnet101, ResNet101_Weights
import torchvision        


img_encoder = torchvision.models.resnet101(
            weights=ResNet101_Weights.IMAGENET1K_V2
        )  # shape out = 512 for resnet 18

# hook the feature extractor 

/home/ptthang/gaze_sample/env_torchism/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def image_encoder_forward(x):
    # resnet forward
    x = img_encoder.conv1(x)
    x = img_encoder.bn1(x)
    x = img_encoder.relu(x)
    x = img_encoder.maxpool(x)

    x = img_encoder.layer1(x)
    x = img_encoder.layer2(x)
    x = img_encoder.layer3(x)
    x = img_encoder.layer4(x)
    return x

In [5]:
import inspect
print(inspect.getsource(img_encoder._forward_impl))

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

